## 6-7. 프로젝트: 멋진 작사가 만들기

### [루브릭]
평가문항	상세기준
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
  -텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  
      
      
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
  -특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  
     
     
3. 텍스트 생성모델이 안정적으로 학습되었는가?  
  -텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?  
    
### [학습 과정]
1. 데이터 다운로드
2. 데이터 읽어오기
3. 데이터 정제
4. 평가 데이터셋 분리
5. 인공지능 만들기

### [결과 및 회고]

### 데이터 읽어오기

In [11]:
import re                  
import numpy as np         
import tensorflow as tf 
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/E06_LyricistAI/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 읽어오기
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free', 'They come to join us here', 'From sea to shining sea And they all have a dream', 'As people always will', 'To be safe and warm', 'In that shining city on the hill Some wanna slam the door', 'Instead of opening the gate', "Aw, let's turn this thing around", 'Before it gets too late [Chorus]', "It's up to me and you", 'Love can conquer hate', 'I know this to be true', "That's what makes us great [Verse 2]", "Don't tell me a lie", 'And sell it as a fact', "I've been down that road before", "And I ain't goin' back And don't you brag to me", 'That you never read a book', 'I never put my faith', "In a con man and his crooks I won't follow down that path", 'And tempt the hands of fate', "Aw, let's turn this thing around", 'Before it gets too late [Chorus]', "It's up to me and you", 'Love can conquer hate', 'I know this to be true', "That's what makes us great In the quiet of the night", 'I lie here w

### 데이터 정제하기

In [105]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    #if sentence[0] == "[": continue   # 문장 처음이 '[' 인 문장은 건너뜁니다.
    #if sentence[-1] == "]": continue  # 문장 끝이 ']' 인 문장은 건너뜁니다
    if idx > 30: break   # 문장 30개만 확인
    print(sentence)

[Verse 1]
They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around
Before it gets too late [Chorus]
It's up to me and you
Love can conquer hate
I know this to be true
That's what makes us great [Verse 2]
Don't tell me a lie
And sell it as a fact
I've been down that road before
And I ain't goin' back And don't you brag to me
That you never read a book
I never put my faith
In a con man and his crooks I won't follow down that path
And tempt the hands of fate
Aw, let's turn this thing around
Before it gets too late [Chorus]
It's up to me and you
Love can conquer hate
I know this to be true
That's what makes us great In the quiet of the night
I lie here wide awake
And I ask myself


토큰화

1. 문장 부호 양쪽에 공백을 추가
2. 모든 문자들을 소문자로 변환
3. 특수문자들은 모두 제거

In [106]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)       # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)              # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    if "verse" in sentence:
        sentence = sentence.replace("verse", "")
    if "chorus" in sentence:
        sentence = sentence.replace("chorus", "")
    
    
    return sentence


In [107]:
corpus = []

#raw_corpus = raw_corpus.remove("")
for sentence in raw_corpus:
    if len(sentence) == 0: continue
        
    corpus.append(preprocess_sentence(sentence))
corpus.remove('<start>  <end>')         

corpus[:30]


['<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> in that shining city on the hill some wanna slam the door <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>',
 '<start> before it gets too late  <end>',
 '<start> it s up to me and you <end>',
 '<start> love can conquer hate <end>',
 '<start> i know this to be true <end>',
 '<start> that s what makes us great  <end>',
 '<start> don t tell me a lie <end>',
 '<start> and sell it as a fact <end>',
 '<start> i ve been down that road before <end>',
 '<start> and i ain t goin back and don t you brag to me <end>',
 '<start> that you never read a book <end>',
 '<start> i never put my faith <end>',
 '<start> in a con man and his crooks i won t follow down that pat

In [ ]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

In [ ]:
print(tensor[:3, :10])

In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

In [ ]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

인공지능 만들기

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he")